In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
from scipy.special import erf, erfinv
import gmplot

from matplotlib.lines import Line2D

In [2]:
crises_df = pd.read_csv('data/african_crises.csv')
crises_df = crises_df.replace({'banking_crisis': {'crisis': 1, 'no_crisis': 0}})
print(crises_df.isnull().sum())
print()
for index, col in enumerate(crises_df.columns):
    print(index, col)

case                               0
cc3                                0
country                            0
year                               0
systemic_crisis                    0
exch_usd                           0
domestic_debt_in_default           0
sovereign_external_debt_default    0
gdp_weighted_default               0
inflation_annual_cpi               0
independence                       0
currency_crises                    0
inflation_crises                   0
banking_crisis                     0
dtype: int64

0 case
1 cc3
2 country
3 year
4 systemic_crisis
5 exch_usd
6 domestic_debt_in_default
7 sovereign_external_debt_default
8 gdp_weighted_default
9 inflation_annual_cpi
10 independence
11 currency_crises
12 inflation_crises
13 banking_crisis


In [3]:
import folium 
import json
import copy
import scipy.stats as stats
import math
from IPython.core.display import display, HTML

countriesString = 'world-countries.json'
file = open(countriesString).read()
countries = json.loads(file)
GDPgrowthdata = 'data/GDPgrowth_Worldbank.csv'
startyear = 1961
endyear = 2014

In [4]:
def filterCountries(countries, countrycodes):
    indexes = []
    accepted = []
    for i in range(len(countries['features'])):
        if countries['features'][i]['id'] not in countrycodes:
            indexes += [i]
        else:
            accepted += [countries['features'][i]['id']]
            
    newcountries = copy.deepcopy(countries)
    indexes.sort(reverse=True)
    for index in indexes:
        newcountries['features'].pop(index)
    return newcountries, accepted

countrycodes = {'France': ['BEN', 'BFA', 'CAF', 'CIV', 'CMR', 'COM', 'GAB', 'GIN', 'MDG','MLI', 'MRT', 'NER', 'SEN', 'TCD', 'TGO', 'MAR', 'DZA', 'TUN'], 'GB': ['BWA', 'GHA', 'GMB', 'KEN', 'LSO', 'MUS', 'MWI', 'NGA', 'SDN', 'SLE', 'SSD', 'SWZ', 'SYC', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE','EGY'], 'Belgium': ['BDI', 'COD', 'COG', 'RWA'], 'Italy': ['ERI', 'SOM'], 'Portugal': ['AGO', 'CPV', 'GNB', 'MOZ', 'STP'], 'Spain': ['GNQ'], 'Ind': ['ETH', 'LBR']}
col_wld = ['WLD', 'FRA', 'GBR']
col_France_GB = countrycodes['France'] + countrycodes['GB']
col_France = countrycodes['France']
col_GB = countrycodes['GB']

filteredCountries, accepted = filterCountries(countries, col_France_GB)

In [5]:
def GDPgrowth(AfricanCountriesCode):
    """
    Extracts growth rate per country
    """
    df = pd.read_csv(GDPgrowthdata)
    years = np.arange(startyear, endyear+1).tolist()
    GDPgrowth_percountry = {}
    growth_France = []
    growth_GB = []
    growth_All = []

    for country in ['FRA', 'GBR', 'WLD']:
        df2 = df.loc[df['Country Code'] == country, str(years[0]):str(years[-1])]
        df2 = df2.values[0]
        GDPgrowth_percountry[country] = df2

    for country in AfricanCountriesCode:
        if country in col_France or country in col_GB:
            df3 = df.loc[df['Country Code'] == country, str(years[0]):str(years[-1])]
            df3 = df3.values[0]

            GDPgrowth_percountry[country] = df3

    # categorize growth numbers by colonial overlord:
    nFrance = 0
    nGB = 0
    for country in GDPgrowth_percountry:
        # check if data is complete
        complete_data = True
        for gdpval in GDPgrowth_percountry[country]:
            if str(gdpval) == 'nan':
                complete_data = False

        if complete_data:
            growth_All += GDPgrowth_percountry[country].tolist()
            if country in col_France:
                nFrance += 1
                growth_France += GDPgrowth_percountry[country].tolist()
            elif country in col_GB:
                nGB += 1
                growth_GB += GDPgrowth_percountry[country].tolist()

    # calculate stats to fit normal distribution:
    avg_France = np.mean(growth_France)
    stddev_France = np.std(growth_France)
    avg_GB = np.mean(growth_GB)
    stddev_GB = np.std(growth_GB)
    avg_All = np.mean(growth_All)
    stddev_All = np.std(growth_All)

    statFrance = {'avg': avg_France, 'stddev': stddev_France, 'n_countries': nFrance, 'n_sample': len(growth_France)}
    statGB = {'avg': avg_GB, 'stddev': stddev_GB, 'n_countries': nGB, 'n_sample': len(growth_GB)}
    statAll = {'avg': avg_All, 'stddev': stddev_All, 'n_countries': nGB + nFrance, 'n_sample': len(growth_All)}

    growth_stats = [statFrance, statGB, statAll]
    # print(growth_stats)
    growth_percol = [growth_France, growth_GB, growth_All]

    return GDPgrowth_percountry, growth_percol, growth_stats


def regress_growth(GDPgrowth_percountry):
    """
    Regresses the growth rates of countries
    """
    corr_wld = []
    pval_wld = []
    corr_afr = []
    pval_afr = []

    for country in ['FRA', 'GBR', 'WLD']:
        corr = []
        pval = []
        for Afrcountry in col_France_GB:
            regress = stats.linregress(GDPgrowth_percountry[country], GDPgrowth_percountry[Afrcountry])
            r_value = regress[2]
            p_value = regress[3]
            corr.append(r_value)
            pval.append(p_value)

        corr_wld.append(corr)
        pval_wld.append(pval)


    corr_wld=np.array(corr_wld)
    pval_wld=np.array(pval_wld)

    for Afrcountry1 in col_France_GB:
        corr = []
        pval = []
        for Afrcountry2 in col_France_GB:
            regress = stats.linregress(GDPgrowth_percountry[Afrcountry1], GDPgrowth_percountry[Afrcountry2])
            r_value = regress[2]
            p_value = regress[3]
            corr.append(r_value)
            pval.append(p_value)

        corr_afr.append(corr)
        pval_afr.append(pval)

    corr_afr = np.array(corr_afr)
    pval_afr = np.array(pval_afr)

    return corr_wld, pval_wld, corr_afr, pval_afr

GDPgrowth_percountry, growth_percol, growth_stats = GDPgrowth(col_France_GB)

corr_wld, pval_wld, corr_afr, pval_afr = regress_growth(GDPgrowth_percountry)

In [6]:
def interpetCorrelations(corrs, xcols, ycols):
    data = {}
    for i in range(len(corrs)):
        values = [corr if not math.isnan(corr) else 0 for corr in corrs[i]]
        dictionary = {'Country': ycols, 'corr': values}
        data[xcols[i]] = pd.DataFrame(data=dictionary)
    return data

corrDataAfr = interpetCorrelations(corr_afr, col_France_GB, col_France_GB)
corrDataWld = interpetCorrelations(corr_wld, col_wld, col_France_GB)
print(list(corrDataAfr.keys()))
print()
print(list(corrDataWld.keys()))


['BEN', 'BFA', 'CAF', 'CIV', 'CMR', 'COM', 'GAB', 'GIN', 'MDG', 'MLI', 'MRT', 'NER', 'SEN', 'TCD', 'TGO', 'MAR', 'DZA', 'TUN', 'BWA', 'GHA', 'GMB', 'KEN', 'LSO', 'MUS', 'MWI', 'NGA', 'SDN', 'SLE', 'SSD', 'SWZ', 'SYC', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE', 'EGY']

['WLD', 'FRA', 'GBR']


In [9]:
def specificData(targetData, lst):
    return targetData.query('Country in @lst')

def createHeatMap(corrData, target):
    targetData = corrData[target]
    alignmentData = pd.DataFrame({"Country":['min', 'max'], "corr":[0,1]})

    targetDataGB = specificData(targetData, col_GB)
    targetDataGB = targetDataGB.append(alignmentData, ignore_index=True)

    targetDataFrance = specificData(targetData, col_France)
    targetDataFrance = targetDataFrance.append(alignmentData,ignore_index=True)

    filteredCountriesGB, _ = filterCountries(countries, col_GB)
    filteredCountriesFrance, _ = filterCountries(countries, col_France)


    m = folium.Map(location=[2,17], zoom_start=4, zoomControl=False)
    m.choropleth(
        geo_data=filteredCountriesGB
        ,name='choropleth'
        ,zoom_on_click=False
        ,zoomControl=False
        ,data=targetDataGB
        ,columns=['Country', 'corr']
        ,key_on='feature.id'
        ,fill_color='YlOrRd'
        ,fill_opacity=0.7
         ,line_opacity=0.2)

    m.choropleth(
        geo_data=filteredCountriesFrance
        ,name='choropleth'
        ,zoom_on_click=False
        ,zoomControl=False
        ,data=targetDataFrance
        ,columns=['Country', 'corr']
        ,key_on='feature.id'
        ,fill_color='YlGnBu'
        ,fill_opacity=0.7
         ,line_opacity=0.2)

    filename = 'heatmaps/Heatmap' + str(target) + '.html'
    m.save(filename)
    return m

for target in list(corrDataWld.keys()):
    createHeatMap(corrDataWld, target)
    
for target in list(corrDataAfr.keys()):
    createHeatMap(corrDataAfr, target)
